In [53]:
import requests
from datetime import datetime, timedelta
import time
import os
import alpaca_trade_api as tradeapi
import pandas as pd
from transformers import pipeline, BertTokenizer, BertForSequenceClassification

In [54]:
top_50_sp500_stocks = [
    'AAPL',  # Apple Inc.
    'MSFT',  # Microsoft Corporation
    'AMZN',  # Amazon.com Inc.
    'NVDA',  # NVIDIA Corporation
    'GOOGL', # Alphabet Inc. (Class A)
    'GOOG',  # Alphabet Inc. (Class C)
    'TSLA',  # Tesla Inc.
    'META',  # Meta Platforms Inc.
    'BRK.B', # Berkshire Hathaway Inc. (Class B)
    'UNH',   # UnitedHealth Group Incorporated
    'JNJ',   # Johnson & Johnson
    'XOM',   # Exxon Mobil Corporation
    'V',     # Visa Inc.
    'PG',    # Procter & Gamble Co.
    'JPM',   # JPMorgan Chase & Co.
    'LLY',   # Eli Lilly and Company
    'MA',    # Mastercard Incorporated
    'HD',    # The Home Depot Inc.
    'CVX',   # Chevron Corporation
    'MRK',   # Merck & Co. Inc.
    'PEP',   # PepsiCo Inc.
    'ABBV',  # AbbVie Inc.
    'KO',    # The Coca-Cola Company
    'PFE',   # Pfizer Inc.
    'AVGO',  # Broadcom Inc.
    'COST',  # Costco Wholesale Corporation
    'MCD',   # McDonald's Corporation
    'TMO',   # Thermo Fisher Scientific Inc.
    'WMT',   # Walmart Inc.
    'DHR',   # Danaher Corporation
    'NKE',   # NIKE Inc.
    'DIS',   # The Walt Disney Company
    'ADBE',  # Adobe Inc.
    'NFLX',  # Netflix Inc.
    'VZ',    # Verizon Communications Inc.
    'CSCO',  # Cisco Systems Inc.
    'ABT',   # Abbott Laboratories
    'ACN',   # Accenture plc
    'NEE',   # NextEra Energy Inc.
    'LIN',   # Linde plc
    'TXN',   # Texas Instruments Incorporated
    'MDT',   # Medtronic plc
    'PM',    # Philip Morris International Inc.
    'WFC',   # Wells Fargo & Company
    'HON',   # Honeywell International Inc.
    'QCOM',  # QUALCOMM Incorporated
    'BMY',   # Bristol-Myers Squibb Company
    'LOW',   # Lowe's Companies Inc.
    'UNP',   # Union Pacific Corporation
    'RTX'    # Raytheon Technologies Corporation
]


In [55]:
# Alpaca API credentials
ALPACA_API_KEY = "PKDFQIPQCXE38TAG1WKG"
ALPACA_SECRET_KEY = "LRaWwamsMm0WDa58x8S0z8wje9gRcYhcgtMf7C55"
ALPACA_URL = 'https://paper-api.alpaca.markets'

# Initialize Alpaca API
alpaca = tradeapi.REST(ALPACA_API_KEY, ALPACA_SECRET_KEY, base_url=ALPACA_URL, api_version='v2')

In [9]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load your data
data = pd.read_csv('stock_data.csv')

# Normalize the input features (open, high, low, close, volume, and sentiment score)
scaler = MinMaxScaler()
data[['open', 'high', 'low', 'close', 'volume', 'average_sentiment_score']] = scaler.fit_transform(
    data[['open', 'high', 'low', 'close', 'volume', 'average_sentiment_score']]
)

# Create the target variable (next day's closing price)
data['next_day_close'] = data.groupby('symbol')['close'].shift(-1)

# Drop any rows with NaN values (e.g., the last row for each symbol)
data.dropna(inplace=True)

# Split data into features and target
X = data[['open', 'high', 'low', 'close', 'volume', 'average_sentiment_score']]
y = data['next_day_close']


In [48]:
# load stock data csv into dataframe

stock_data = pd.read_csv('stock_data.csv')

stock_data.head()

,symbol,time,open,high,low,close,volume,date,average_sentiment_score
0,AAPL,2024-07-24 00:00:00-04:00,224.115,224.765,217.165,218.585,1005989,2024-07-24 00:00:00.000000,0.974538
1,AAPL,2024-07-25 00:00:00-04:00,218.880,220.810,214.640,217.420,642703,2024-07-25 00:00:00.000000,0.947577
2,AAPL,2024-07-26 00:00:00-04:00,218.940,219.480,216.040,218.030,661067,2024-07-26 00:00:00.000000,0.972036
3,AAPL,2024-07-29 00:00:00-04:00,217.375,219.280,215.790,218.185,381966,2024-07-29 00:00:00.000000,0.955637
4,AAPL,2024-07-30 00:00:00-04:00,219.300,220.270,216.120,218.680,559407,2024-07-30 00:00:00.000000,0.964756


In [56]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
import pandas as pd
from datetime import datetime

def trading_strategy(symbols, file_path='stock_data.csv', max_investment_ratio=0.1):
    """
    Trading strategy to decide whether to buy, sell, hold, or not buy based on the predicted price.
    """
    # Load the stock data
    stock_data = pd.read_csv(file_path)

    # Convert 'time' column to datetime
    stock_data['time'] = pd.to_datetime(stock_data['time'])

    # Get today's date
    today_date = datetime.utcnow().date()

    decisions = []

    for symbol in symbols:
        # Filter data for the given symbol
        symbol_data = stock_data[stock_data['symbol'] == symbol]

        # Prepare input features (X) and labels (y)
        X = symbol_data[['open', 'high', 'low', 'close', 'volume', 'average_sentiment_score']].values
        y = symbol_data['close'].shift(-1).dropna().values  # Next day's closing price as the target

        # Adjust X to match the length of y
        X = X[:-1]

        # Check the shape of X
        print(f"Symbol: {symbol}, X shape: {X.shape}, Expected features: 6")

        # Split the data into training and testing sets (using all data for training)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Scale the features
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Train a neural network model
        model = MLPRegressor(hidden_layer_sizes=(50, 50), max_iter=1000, random_state=42)
        model.fit(X_train, y_train)

        # Predict the next day's price using the latest available data
        latest_data = symbol_data[symbol_data['time'].dt.date == today_date]

        if latest_data.empty:
            print(f"No data found for {symbol} on {today_date}.")
            decisions.append((symbol, None, None, None))
            continue

        current_price = latest_data['close'].values[0]

        # Prepare the input features for prediction
        X_latest = latest_data[['open', 'high', 'low', 'close', 'volume', 'average_sentiment_score']].values.reshape(1, -1)

        # Check the shape of X_latest
        print(f"Symbol: {symbol}, X_latest shape: {X_latest.shape}, Expected features: 6")

        X_latest = scaler.transform(X_latest)

        predicted_price = model.predict(X_latest)[0]

        # Decision-making logic
        if predicted_price > current_price * 1.02:
            decision = 'buy'
        elif predicted_price < current_price * 0.98:
            decision = 'sell'
        else:
            decision = 'hold'

        decisions.append((symbol, decision, current_price, predicted_price))

    return decisions


In [5]:
# Function to execute trades with hedging
def execute_trade(symbol, decision, hedge_sell_ratio=0.5, max_investment_ratio=0.1):
    account = alpaca.get_account()
    cash = float(account.cash)
    
    # Check if there's an existing position in the symbol
    position = alpaca.get_position(symbol) if symbol in [p.symbol for p in alpaca.list_positions()] else None
    current_qty = float(position.qty) if position else 0

    if decision == 'buy':
        # Calculate how much to buy based on available cash and hedging
        qty_to_buy = (cash * max_investment_ratio) / current_price
        alpaca.submit_order(
            symbol=symbol,
            qty=qty_to_buy,
            side='buy',
            type='market',
            time_in_force='day'
        )
    elif decision == 'sell' and current_qty > 0:
        # Calculate how much to sell, hedging by selling only a portion of current holdings
        qty_to_sell = current_qty * hedge_sell_ratio
        alpaca.submit_order(
            symbol=symbol,
            qty=qty_to_sell,
            side='sell',
            type='market',
            time_in_force='day'
        )

In [12]:
# Example: Adjusting hedging ratios
hedge_sell_ratio = 0.5  # Sell 50% of current holdings when selling
max_investment_ratio = 0.1  # Invest up to 10% of available cash when buying


In [58]:
trading_strategy(top_50_sp500_stocks)
    
    

Symbol: AAPL, X shape: (22, 6), Expected features: 6


/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


No data found for AAPL on 2024-08-23.
Symbol: MSFT, X shape: (22, 6), Expected features: 6


/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


No data found for MSFT on 2024-08-23.
Symbol: AMZN, X shape: (22, 6), Expected features: 6


...

No data found for GOOG on 2024-08-23.
Symbol: TSLA, X shape: (22, 6), Expected features: 6


/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


No data found for TSLA on 2024-08-23.
Symbol: META, X shape: (22, 6), Expected features: 6
No data found for META on 2024-08-23.
Symbol: BRK.B, X shape: (22, 6), Expected features: 6


/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/utils/extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/opt/conda/lib/python3.11/site-packages/sklearn/utils/extmath.py:1055: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/opt/conda/lib/python3.11/site-packages/sklearn/utils/extmath.py:1075: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


ValueError: Input X contains NaN.
MLPRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values